# KITTI Odometry Dataset

## Import

In [2]:
import os
from io import TextIOWrapper
import sys
import struct
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.transform import Rotation
from pathlib import Path
from glob import glob
import cv2
KITTI_ROOT = Path('/data/datasets/dataset_kitti')
KITTI_SEQS = ['00', '01', '02',
              '03', '04', '05', 
              '06', '07', '08',
              '09', '10']

## Rearrage LiDAR

In [3]:
def rearrage_calib(root:Path, seq:str):
    old_calib_fn = root / 'data_odometry_calib' / 'dataset' / 'sequences' / seq / 'calib.txt'
    old_times_fn = root / 'data_odometry_calib' / 'dataset' / 'sequences' / seq / 'times.txt'
    new_calib_fn = root / seq / 'raw_calib.txt'
    new_times_fn = root / seq / 'raw_times.txt'
    if new_calib_fn.exists() :
        return False
    if new_times_fn.exists():
        return False
    
    # Rename
    os.rename(old_calib_fn, new_calib_fn)
    os.rename(old_times_fn, new_times_fn)
    return True

def rearrage_poses(root:Path, seq:str):
    old_poses_fn = root / 'data_odometry_poses' / 'dataset' / 'poses' / f'{seq}.txt'
    new_poses_fn = root / seq / 'raw_poses.txt'
    if new_poses_fn.exists():
        return False
    
    os.rename(old_poses_fn, new_poses_fn)
    return True

def rearrage_lidar(root:Path, seq:str):
    old_lidar_dir = root / seq / 'velodyne'
    new_lidar_dir = root / seq / 'lidar'
    old_paths = glob(f'{old_lidar_dir}/*.bin')
    new_paths = glob(f'{new_lidar_dir}/*.bin')
    old_N = len(old_paths)
    new_N = len(new_paths)
    if new_N != 0:
        print('Lidar is already re-arranged')
        return False
    
    old_paths.sort()
    os.makedirs(new_lidar_dir, exist_ok=True)
    for old_path in old_paths:
        old_fn = Path(old_path)
        new_fn = new_lidar_dir / old_fn.name
        print(new_fn)
        os.rename(old_fn, new_fn)
        
    return True

def rearrage_data(root:Path, seq:str):
    if not root.exists():
        print(f"{root} doesn't exist!")
        return False
    
    if not (root / seq ).exists():
        os.makedirs(root/seq)

    rearrage_calib(root, seq)
    rearrage_poses(root, seq)
    rearrage_lidar(root, seq)
    calib_fn = root / seq / 'raw_calib.txt'
    times_fn = root / seq / 'raw_times.txt'
    poses_fn = root / seq / 'raw_poses.txt'
    # calib = np.genfromtxt(calib_fn, dtype=float)
    times = np.genfromtxt(times_fn, dtype=float)
    poses = np.genfromtxt(poses_fn, dtype=float)
    print('timestamps: ', times.shape)
    print('poses: ', poses.shape)
    
    lidar_dir = root / seq / 'lidar'
    lidar_fns = glob(f'{lidar_dir}/*.bin')
    lidar_fns.sort()
    N_SCANS = len(lidar_fns)
    print(f"N_SCANS: {N_SCANS}")
    
    assert (N_SCANS == times.shape[0])
    assert (N_SCANS == poses.shape[0])
    
    data_pose_fn = root / seq / 'pose.txt'
    data_pose_csv_fn = root / seq / 'pose.csv'
    f_pose = open(data_pose_fn, 'w')
    f_pose_csv = open(data_pose_csv_fn, 'w')
    for i in range(N_SCANS):
        time = times[i]
        pose = poses[i]
        rot_mat = np.array([pose[0:3],
                            pose[4:7],
                            pose[8:11]])
        # print(rot_mat.shape)
        rot_mat = Rotation.from_matrix(rot_mat)
        quat = rot_mat.as_quat() # x, y, z, w
        # print(quat.shape)
        trans = pose[3::4]
        # print(trans.shape)
        line = f"{time:.6f} {trans[0]:.6f} {trans[1]:.6f} {trans[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n"
        f_pose.write(line)
    f_pose.close()
    f_pose_csv.close()
    
    lidar_data_fn = root / seq / 'lidar_data.txt'
    f_lidar = open(lidar_data_fn, 'w')
    for i in range(N_SCANS):
        time = times[i]
        lidar_fn = Path(lidar_fns[i])
        line = f"{time:.6f} lidar/{lidar_fn.name}\n"
        f_lidar.write(line)
    f_lidar.close()
    
    return True
    

In [3]:
def rotmat2quat(R):
    rotation = Rotation.from_matrix(R)
    quaternion = rotation.as_quat()
    return quaternion

def procCalib(root: Path, seq: str):
    calib_dir = root / seq / 'calibration'
    if not calib_dir.exists():
        print(f"mkdir {calib_dir}")
        os.makedirs(calib_dir)
    
    old_calib_fn = root / seq / 'raw_calib.txt'
    with open(old_calib_fn, 'r') as f_old_calib:
        lines = f_old_calib.readlines()
    
    calib_cam0_fn = calib_dir / 'ProjMatCam0.txt'
    calib_cam1_fn = calib_dir / 'ProjMatCam1.txt'
    calib_cam2_fn = calib_dir / 'ProjMatCam2.txt'
    calib_cam3_fn = calib_dir / 'ProjMatCam3.txt'
    calib_lidar_fn = calib_dir / 'Base2Lidar.txt'
    for i in range(len(lines)):
        lines[i] = lines[i][3:].strip().split(' ')
        for j in range(len(lines[i])):
            lines[i][j] = '%.6f' % float(lines[i][j])
        # print(lines[i])
    
    with open(calib_cam0_fn, 'w') as fout:
        new_line = ' '.join(lines[0]) + '\n'
        # print(new_line)
        fout.write(new_line)
    with open(calib_cam1_fn, 'w') as fout:
        new_line = ' '.join(lines[1]) + '\n'
        # print(new_line)
        fout.write(new_line)
    with open(calib_cam2_fn, 'w') as fout:
        new_line = ' '.join(lines[2]) + '\n'
        # print(new_line)
        fout.write(new_line)
    with open(calib_cam3_fn, 'w') as fout:
        new_line = ' '.join(lines[3]) + '\n'
        # print(new_line)
        fout.write(new_line)
    with open(calib_lidar_fn, 'w') as fout:
        rotmat = np.zeros((3, 3), dtype=float)
        rotmat[0,0] = float(lines[4][0])
        rotmat[0,1] = float(lines[4][1])
        rotmat[0,2] = float(lines[4][2])
        rotmat[1,0] = float(lines[4][0+4])
        rotmat[1,1] = float(lines[4][1+4])
        rotmat[1,2] = float(lines[4][2+4])
        rotmat[2,0] = float(lines[4][0+8])
        rotmat[2,1] = float(lines[4][1+8])
        rotmat[2,2] = float(lines[4][2+8])
        # rotmat[0,1] = -1.0
        # rotmat[1,2] = -1.0
        # rotmat[2,0] =  1.0
        quat = rotmat2quat(rotmat)
        tr = np.zeros((3,), dtype=float)
        tr[0] = lines[4][3]
        tr[1] = lines[4][7]
        tr[2] = lines[4][11]
        # tr[0] = -0.27
        # tr[1] = 0.0
        # tr[2] = 0.08
        new_line = f"{tr[0]:.6f} {tr[1]:.6f} {tr[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n"
        # print(new_line)
        fout.write(new_line)
    
ROOT = Path('/data/datasets/dataset_kitti')
SEQS = ['00', '01', '02',
        '03', '04', '05', 
        '06', '07', '08',
        '09', '10']
for seq in SEQS:
    procCalib(Path('/data/datasets/dataset_kitti'), seq)

In [4]:
# ROOT = Path('/data/datasets/dataset_kitti')
# SEQS = ['00', '01', '02',
#         '03', '04', '05', 
#         '06', '07', '08',
#         '09', '10']
# for seq in SEQS:
#     rearrage_data(ROOT, seq)

In [12]:
ROOT = Path('/data/datasets/dataset_kitti')
SEQS = ['00', '01', '02',
        '03', '04', '05', 
        '06', '07', '08',
        '09', '10']
seq = SEQS[0]
num = 250




image_fn = ROOT / seq / 'image_0' / f'{num:06d}.png'

Tr_velo_to_cam = np.zeros((4, 4), dtype=float)
Tr_velo_to_cam[0,0] = 7.533745e-03
Tr_velo_to_cam[0,1] = -9.999714e-01
Tr_velo_to_cam[0,2] = -6.166020e-04
Tr_velo_to_cam[0,3] = -4.069766e-03
Tr_velo_to_cam[1,0] = 1.480249e-02
Tr_velo_to_cam[1,1] = 7.280733e-04
Tr_velo_to_cam[1,2] = -9.998902e-01
Tr_velo_to_cam[1,3] = -7.631618e-02
Tr_velo_to_cam[2,0] = 9.998621e-01
Tr_velo_to_cam[2,1] = 7.523790e-03
Tr_velo_to_cam[2,2] = 1.480755e-02
Tr_velo_to_cam[2,3] = -2.717806e-01
Tr_velo_to_cam[3,3] = 1.0
print("Tr_velo_to_cam\n", Tr_velo_to_cam)

# 7.533745000000e-03 -9.999714000000e-01 -6.166020000000e-04 -4.069766000000e-03 1.480249000000e-02 7.280733000000e-04 -9.998902000000e-01 -7.631618000000e-02 9.998621000000e-01 7.523790000000e-03 1.480755000000e-02 -2.717806000000e-01

R0 = np.eye(4, dtype=float)
R0[0,0] = 9.999239e-01
R0[0,1] = 9.837760e-03
R0[0,2] = -7.445048e-03
R0[1,0] = -9.869795e-03
R0[1,1] = 9.999421e-01
R0[1,2] = -4.278459e-03
R0[2,0] = 7.402527e-03
R0[2,1] = 4.351614e-03
R0[2,2] = 9.999631e-01
print("R0\n", R0)

P0 = np.zeros((3, 4), dtype=float)
P0[0,0] = 7.188560000000e+02; P0[0,1] = 0.000000000000e+00; P0[0,2] = 6.071928000000e+02; P0[0,3] = 0.000000000000e+00 
P0[1,0] = 0.000000000000e+00; P0[1,1] = 7.188560000000e+02; P0[1,2] = 1.852157000000e+02; P0[1,3] = 0.000000000000e+00 
P0[2,0] = 0.000000000000e+00; P0[2,1] = 0.000000000000e+00; P0[2,2] = 1.000000000000e+00; P0[2,3] = 0.000000000000e+00
print("P0\n", P0)

# P2 = np.zeros((3, 4), dtype=float)
# P2[0,0] = 7.188560000000e+02; P2[0,1] = 0.000000000000e+00; P2[0,2] = 6.071928000000e+02; P2[0,3] = 4.538225000000e+01 
# P2[1,0] = 0.000000000000e+00; P2[1,1] = 7.188560000000e+02; P2[1,2] = 1.852157000000e+02; P2[1,3] = -1.130887000000e-01 
# P2[2,0] = 0.000000000000e+00; P2[2,1] = 0.000000000000e+00; P2[2,2] = 1.000000000000e+00; P2[2,3] = 3.779761000000e-03
# print("P2\n", P2)

P = P0

# 7.188560000000e+02 0.000000000000e+00 6.071928000000e+02 4.538225000000e+01 0.000000000000e+00 7.188560000000e+02 1.852157000000e+02 -1.130887000000e-01 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 3.779761000000e-03

print(image_fn)
img = cv2.imread(str(image_fn))
img1 = cv2.imread(str(image_fn))

scan_fn = ROOT / seq / 'lidar' / f'{num:06d}.bin'
f_scan = open(scan_fn, 'br')
scan = []
data = f_scan.read(4)
while len(data) == 4:
    val = struct.unpack('f', data)[0]
    scan.append(val)
    data = f_scan.read(4)
f_scan.close()
scan = np.asarray(scan, dtype=float).reshape((int(len(scan)/4), 4))
# print("scans: \n", scan[:, :10])
# scan = np.asarray(scan, dtype=float).reshape((4, int(len(scan)/4)))

XYZ1 = np.hstack((scan[:,0:3], np.ones((scan.shape[0], 1), dtype=float))).transpose()

# print(img.shape)
# print("XYZ1 length: ", XYZ1.shape)
# print("P.shape: ", P.shape)
# print("R0.shape: ", R0.shape)
# print("Tr_velo_to_cam.shape: ", Tr_velo_to_cam.shape)
# print("XYZ1.shape: ", XYZ1.shape)

print("XYZ1: \n", XYZ1[:, :5])

xy1 = Tr_velo_to_cam @ XYZ1
print("xy1: \n", xy1[:, :5])

xy1 = R0 @ xy1
print("world xy1: \n", xy1[:, :5])

xy1 = P @ xy1
print("pixel xy1: \n", xy1[:, :5])


# xy1 = P2 @ R0 @ Tr_velo_to_cam @ XYZ1
# print("xy1.shape: ", xy1.shape)
# print(xy1[:, :10])

xy1 = xy1[:, (xy1[2,:] > 0)]
print("xy1.shape: ", xy1.shape)
xy1 = xy1[0:2,:] / xy1[2,:]
print("xy1.shape: ", xy1.shape)

N_FORWARD_SCAN = xy1.shape[1]
# print(img.shape)
w = img.shape[1]
h = img.shape[0]

for i in range(N_FORWARD_SCAN):
    pt = (int(xy1[0, i] + 0.5), int(xy1[1, i] + 0.5))
    
    if pt[0] < 0 or w <= pt[0]:
        continue
    if pt[1] < 0 or h <= pt[1]:
        continue
     
    print(pt)
    
    cv2.circle(img, pt, 1, (0, 255, 0))
        
cv2.imshow('img', img)
cv2.imshow('img1', img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

Tr_velo_to_cam
 [[ 7.533745e-03 -9.999714e-01 -6.166020e-04 -4.069766e-03]
 [ 1.480249e-02  7.280733e-04 -9.998902e-01 -7.631618e-02]
 [ 9.998621e-01  7.523790e-03  1.480755e-02 -2.717806e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
R0
 [[ 0.9999239   0.00983776 -0.00744505  0.        ]
 [-0.0098698   0.9999421  -0.00427846  0.        ]
 [ 0.00740253  0.00435161  0.9999631   0.        ]
 [ 0.          0.          0.          1.        ]]
P0
 [[718.856    0.     607.1928   0.    ]
 [  0.     718.856  185.2157   0.    ]
 [  0.       0.       1.       0.    ]]
/data/datasets/dataset_kitti/00/image_0/000250.png
XYZ1: 
 [[73.44501495 73.16756439 70.59056854 36.47029495 36.58885193]
 [ 1.54282212  1.76679623  2.70268822  5.37737322  5.51235819]
 [ 2.6933105   2.68435621  2.59854913  1.45705581  1.46208608]
 [ 1.          1.          1.          1.          1.        ]]
xy1: 
 [[-0.99519245 -1.22124487 -2.17647161 -5.10742971 -5.24152075]
 [-1.68103857 -1.67602915 -1.62769